In [8]:
%reload_ext autoreload
%autoreload 2

In [23]:
import tensorflow as tf

from coffeedd.ml_logic.data import create_dataset_from_directory, create_tf_dataset
from coffeedd.ml_logic.model import initialize_model, compile_model, train_model, evaluate_model
from coffeedd.ml_logic.custom_weights import get_class_weights
from coffeedd.params import CLASS_NAMES, MODELS_PATH, SAMPLE_SIZE, LOCAL_DATA_PATH, BATCH_SIZE, FINE_TUNE


Loading TensorFlow...

✅ TensorFlow loaded (0.0s)

Loading TensorFlow...

✅ TensorFlow loaded (0.0s)


In [12]:
print("🚀 Cargando datos...")
(train_paths, train_labels), (val_paths, val_labels), (test_paths, test_labels) = \
    create_dataset_from_directory(LOCAL_DATA_PATH, CLASS_NAMES, sample_size=SAMPLE_SIZE)

# Crear datasets de TensorFlow
train_dataset = create_tf_dataset(train_paths, train_labels, BATCH_SIZE,
                                  is_training=True, augment=True)
val_dataset = create_tf_dataset(val_paths, val_labels, BATCH_SIZE,
                                is_training=False, augment=False)
test_dataset = create_tf_dataset(test_paths, test_labels, BATCH_SIZE,
                                 is_training=False, augment=False)

print("\n✅ Datasets creados exitosamente (carga on-the-fly activada)")

🚀 Cargando datos...
📂 Escaneando directorio...
  healthy: 18983 imágenes
  cerscospora: 7681 imágenes
  leaf_rust: 8201 imágenes
  miner: 16978 imágenes
  phoma: 6571 imágenes

✅ Total disponible: 58414 imágenes

🎲 Aplicando muestreo...
  📉 Reducido de 58414 a 14603 imágenes
  📊 Distribución muestreada:
    healthy: 4746 (32.5%)
    cerscospora: 1920 (13.1%)
    leaf_rust: 2050 (14.0%)
    miner: 4244 (29.1%)
    phoma: 1643 (11.3%)

📊 División de datos:
  Train: 10223 (70.0%)
  Val:   2190 (15.0%)
  Test:  2190 (15.0%)

📈 Distribución por clase:

  Train:
    healthy             :  3284 ( 32.1%)
    cerscospora         :  1346 ( 13.2%)
    leaf_rust           :  1438 ( 14.1%)
    miner               :  3017 ( 29.5%)
    phoma               :  1138 ( 11.1%)

  Val:
    healthy             :   724 ( 33.1%)
    cerscospora         :   274 ( 12.5%)
    leaf_rust           :   328 ( 15.0%)
    miner               :   605 ( 27.6%)
    phoma               :   259 ( 11.8%)

  Test:
    health

In [21]:
class_weights = get_class_weights(
    train_labels=train_labels,
)


⚖️  CALCULANDO CLASS WEIGHTS DINÁMICOS

🎯 Configuración optimizada para DETECCIÓN DE ENFERMEDADES:
  📊 Objetivo: Minimizar falsos negativos (enfermedad → healthy)
  🦠 Clases de enfermedad detectadas: 4 (cerscospora, leaf_rust, miner, phoma)
  🌱 Clase healthy: Será penalizada para evitar falsos negativos
  📈 Métrica principal: Recall
  ⏰ Epochs configurados: 50

⏳ Class weights se calcularán después de cargar los datos...

📈 Análisis de distribución de clases en dataset completo:
  🦠 cerscospora    :  7681 muestras ( 77.9% vs promedio)
  🦠 leaf_rust      :  8201 muestras ( 83.2% vs promedio)
  🦠 miner          : 16978 muestras (172.2% vs promedio)
  ⚠️ phoma          :  6571 muestras ( 66.7% vs promedio)

  📉 Clase más pequeña: phoma (6571 muestras)
  📊 Promedio por clase: 9857.8 muestras
  🔢 Ratio vs promedio: 0.67
  🔢 Ratio vs máxima: 0.39
  ✅ Distribución relativamente balanceada - No se requiere boost especial

📊 Distribución en train set (10223 muestras):
  healthy        : 3284 (

In [13]:
model, useefficientnet = initialize_model(train_labels)


🏗️  Construyendo modelo...
📊 Dataset grande detectado (10223 imágenes)
🔧 Usando EfficientNetB0 con transfer learning
✅ Modelo inicializado
🏷️  Modelo seleccionado: EfficientNetB0


In [14]:
compile_model(model)

✅ Modelo compilado

📋 Resumen del modelo:
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 avg_pool (GlobalAveragePool  (None, 1280)             0         
 ing2D)                                                          
                                                                 
 batch_normalization_6 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense) 

In [ ]:
model, history = train_model(
    model,
    train_dataset,
    train_labels,
    val_dataset,
    val_labels,
    class_weights=class_weights,
    use_efficientnet=useefficientnet,
    fine_tune=FINE_TUNE
)


🚂 FASE 1: Entrenando con EfficientNet congelado (15 epochs)
Epoch 1/15


2025-10-28 22:37:30.027514: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


320/320 [==============================] - ETA: 0s - loss: 0.6060 - accuracy: 0.7682 - recall: 0.7267 - precision: 0.8164 - disease_recall: 0.6664 - auc: 0.9537
Epoch 1: val_recall improved from -inf to 0.27626, saving model to /Users/fernandorios/code/fermx3/coffee-disease-detection/data/models/best_model_EfficientNetB0_2190.keras
320/320 [==============================] - 110s 335ms/step - loss: 0.6060 - accuracy: 0.7682 - recall: 0.7267 - precision: 0.8164 - disease_recall: 0.6664 - auc: 0.9537 - val_loss: 1.9599 - val_accuracy: 0.2763 - val_recall: 0.2763 - val_precision: 0.2763 - val_disease_recall: 0.4127 - val_auc: 0.5482 - lr: 1.0000e-04
Epoch 2/15
320/320 [==============================] - ETA: 0s - loss: 0.4364 - accuracy: 0.8295 - recall: 0.8051 - precision: 0.8502 - disease_recall: 0.7498 - auc: 0.9750
Epoch 2: val_recall improved from 0.27626 to 0.70091, saving model to /Users/fernandorios/code/fermx3/coffee-disease-detection/data/models/best_model_EfficientNetB0_2190.kera

In [ ]:
evaluate_model(
    model,
    test_dataset,
    test_labels,
    useefficientnet=useefficientnet,
    get_confusion_matrix=False,
    get_false_negatives_analysis=True
)


🧪 EVALUACIÓN FINAL EN TEST SET
69/69 [==============================] - 20s 282ms/step - loss: 0.1184 - accuracy: 0.9594 - recall: 0.9571 - precision: 0.9637 - disease_recall: 0.9387 - auc: 0.9975

Test Loss: 0.1184
Test Accuracy: 0.9594
Test Recall: 0.9637
69/69 [==============================] - 19s 272ms/step

Clases presentes en test set: [0 1 2 3 4]

📊 CLASSIFICATION REPORT
              precision    recall  f1-score   support

     healthy     1.0000    1.0000    1.0000       738
 cerscospora     0.9105    0.9500    0.9299       300
   leaf_rust     0.9894    0.9894    0.9894       284
       miner     0.9383    0.9534    0.9458       622
       phoma     0.9148    0.8293    0.8699       246

    accuracy                         0.9594      2190
   macro avg     0.9506    0.9444    0.9470      2190
weighted avg     0.9593    0.9594    0.9590      2190


⚠️  ANÁLISIS DETALLADO DE FALSOS NEGATIVOS

⚠️  Falsos Negativos (Enfermedad → Healthy):
  cerscospora         : 0/300 (0.0%)
 

[0.11838815361261368,
 0.9593607187271118,
 0.9570776224136353,
 0.9636781811714172,
 0.9387052059173584,
 0.9975118637084961]